## 2. 그래프 신경망 기본

### 2.1 그래프 신경망의 구조

> - **`그래프 신경망`**은 **그래프**와 **정점의 속성 정보**를 입력으로 받는다.
> - 그래프의 **인접 행렬을 $A$**라고 하자.
>- 인접 행렬 $A$는 $|V| \times |V|$의 이진 행렬이다.
> - 각 **정점 $u$의 속성(Attribute) 벡터**를 $X_u$라고 하자.
> - 정점 속성 벡터 $X_u$는 $m$차원 벡터이고, $m$은 속성의 수를 의미한다.
> - **정점의 속성**의 예시는 다음과 같다.
    - 온라인 소셜 네트워크에서 사용자의 지역, 성별, 연령, 프로필 사진 등 
    - 논문 인용 그래프에서 논문에 사용된 키워드에 대한 원-핫 벡터 
    - PageRank 등의 정점 중심성, 군집 계수(Clustering Coefficient) 등

> - 그래프 신경망은 **이웃 정점들의 정보를 집계하는 과정을 반복**하여 **임베딩**을 얻는다.
> - 대상 정점의 임베딩을 얻기 위해 이웃들 그리고 이웃의 이웃들의 정보를 집계한다.

> - 각 집계 단계를 **층(Layer)**이라고 부르고, 각 층마다 임베딩을 얻는다.
> - 각 층에서는 **이웃들의 이전 층 임베딩을 집계**하여 새로운 임베딩을 얻는다.
> - 0번 층, 즉 입력 층의 임베딩으로는 **정점의 속성 벡터**를 사용한다.

> - 대상 정점 마다 집계되는 정보가 상이하다.
> - 대상 정점 별 집계되는 구조를 **계산 그래프(Computation Graph)**라고 부른다.
> - 서로 다른 대상 정점간에도 **층 별 집계 함수는 공유**한다.

> - 집계 함수는 **(1) 이웃들 정보의 평균을 계산하고 (2) 신경망에 적용**하는 단계를 거친다.
> - $h_v^0 = x_v$
    - 0번 층에서 정점 $v$의 임베딩으로 정점 $v$의 속성 벡터로 초기화
> - $h_v^k = \sigma \left( W_k \sum_{u \in N(v)} \frac{h_u^{k-1}}{|N(v)|} + B_k h_v^{k-1} \right), \forall k \in \{1, \dots, K\}$
> - **마지막 층에서의 정점 별 임베딩**이 해당 정점의 **출력 임베딩**이다.
> - $z_v = h_v^K$

### 2.2 그래프 신경망의 학습

> - 그래프 신경망의 **학습 변수(Trainable Parameter)**는 **층 별 신경망의 가중치**이다.
> - 먼저 **`손실함수`**를 결정한다.
    - **정점간 거리를 보존**하는 것을 목표로 할 수 있다.
> - 변환식 정점 임베딩에서처럼 **그래프에서의 정점간 거리를 보존**하는 것을 목표로 할 수 있다.
> - 만약, 인접성을 기반으로 유사도를 정의한다면, 손실 함수는 다음과 같다.
> - $L = \sum_{(u, v) \in V \times V} ||z_u^\top z_v - A_{u, v}||^2$

> - **후속 과제(Downstream Task)의 손실함수를 이용한 종단종(End-to-End) 학습도 가능**하다.
> - 정점 분류가 최종 목표인 경우를 생각해보자.
    - 예를 들어
    - (1) 그래프 신경망을 이용하여 정점의 임베딩을 얻고
    - (2) 이를 분류기(Classifier)의 입력으로 사용하여
    - (3) 각 정점의 유형을 분류하려고 힌다.
> - 이 경우 분류기의 손실함수, 예를 들어 교차 엔트로피(Cross Entropy)를, 전체 프로세스의 손실함수로 사용하여 종단종(End-to-End) 학습을 할 수 있다.

> - **그래프 신경망의 종단종(End-to-End) 학습을 통한 분류**는, **변환적 정점 임베딩 이후에 별도의 분류기를 학습하는 것**보다 정확도가 대체로 높다.

> - 학습에 사용할 대상 정점을 결정하여 학습 데이터를 구성한다.
> - 선택한 대상 정점들에 대한 계산 그래프를 구성한다.
> - **오차역전파(Backpropagation)**을 통해 손실함수를 최소화한다.
    - 구체적으로, 오차역전파를 통해 신경망의 학습 변수들을 학습한다.
> - 학습된 신경망을 적용하여, 학습에 사용되지 않은 정점의 임베딩을 얻을 수 있다.

### 2.3 그래프 신경망의 활용

> - 학습 이후에 **추가된 정점의 임베딩**도 얻을 수 있다.
> - 온라인 소셜네트워크 등 많은 실제 그래프들은 시간에 따라서 변화한다.

> - 학습된 그래프 신경망을, **새로운 그래프에 적용**할 수도 있다.
    - 예를 들어, A종의 단백질 상호 작용 그래프에서 학습한 그래프 신경망을 B종의 단백질 상호작용 그래프에 적용할 수 있다.
    
## 3. 그래프 신경망 변형

### 3.1 그래프 합성곱 신경망

> - **그래프 합성곱 신경망(Graph Convolutional Network, GCN)**의 **집계 함수**이다.
> - $h_v^0 = x_v$
> - $h_v^k = \sigma \left( W_k \sum_{u \in N(v) \cup v} \frac{h_u^{k-1}}{\sqrt{|N(u)||N(v)|}} \right), \forall k \in \{1, \dots, K\}$
> - $z_v = h_v^K$
> - 기존의 집계 함수와 비교해볼때 작은 차이지만 큰 성능의 향상으로 이어지기도 한다.
    - 동일 신경망 사용으로 학습 변수를 공유한다.
    - 정규화 방법의 변화가 있다.

### 3.2 GraphSAGE

> - **GraphSAGE**의 **집계 함수**이다.
> - 이웃들의 임베딩을 **AGG 함수**를 이용해 합친 후, **자신의 임베딩과 연결(Concatenation)**하는 점이 독특하다.
> - $h_v^k = \sigma \left( [W_k \cdot \mathrm{AGG} (\{h_u^{k-1}, \forall u \in N(v) \}), B_k h_v^{k-1}] \right)$

> - **AGG 함수**로는 평균, 풀링, LSTM 등이 사용될 수 있다.
> - **Mean** : $\mathrm{AGG} = \sum_{u \in N(v)} \frac{h_u^{k-1}}{|N(v)|}$
> - **Pool** : $\mathrm{AGG} = \gamma \left( \{ Qh_u^{k-1}, \forall u \in N(v) \} \right)$
> - **LSTM** : $\mathrm{AGG} = \mathrm{LSTM} \left([h_u^{k-1}, \forall u \in \pi N(v)]\right)$

## 4. 합성곱 신경망과의 비교

### 4.1 합성곱 신경망과 그래프 신경망의 유사성

> - **합성곱 신경망**과 **그래프 신경망**은 모두 이웃의 정보를 집계하는 과정을 반복한다.
> - 구체적으로, 합성곱 신경망은 이웃 픽셀의 정보를 집계하는 과정을 반복한다.

### 4.2 합성곱 신경망과 그래프 신경망의 차이

> - **합성곱 신경망**에서는 이웃의 수가 균일하지만, **그래프 신경망**에서는 아니다.
> - 그래프 신경망에서는 정점 별로 집계하는 이웃의 수가 다르다.